In [39]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 

In [64]:
scp_main_df = pd.DataFrame(columns=['code', 'title', 'text', 'link', 'rating'], index=None)

In [42]:
http = httplib2.Http()

In [ ]:
status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
soup = BeautifulSoup(response, 'html.parser')

# note from 2235 onwards there are a couple that do not have the standard codes in directory pages, will keep the standard code for now
# may go back and adjust 

#note need to rejoin the text into a single string maybe with newlines, currently a list of each p tag contents
# also 5257 and 5258 actually seem to not be working atm but placeholder text used on site prob added instead

# the page links all follow pattern and can loop through nums to string just zero padded basically
for n in range(6900,7000):
  # timeout to limit load on the site
  if n % 240 == 0:
    print(n)
    time.sleep(60)
  num_as_str = '{}{}'.format('0' * (3 - len(str(n))), str(n)) if n < 1000 else str(n)

  url = 'https://scp-wiki.wikidot.com/scp-{}'.format(num_as_str)
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')
  text = [i for i in soup.find(id = 'page-content').find_all('p')]
  # p tags include things like image captions may want to exclude, can do with scp-image-caption class there and check other cases
  for i in text:
    if hasattr(i.parent, 'class'):
      if i.parent.get("class") == 'scp-image-caption':
        text.remove(i)
  text = [i.text for i in text]
  rating = soup.find(class_ = 'number prw54353')
  if rating == None:
    pass
  else:
    rating = rating.text
  # below needs to accoun for past 999
  if n < 1000:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
  else:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series-{}'.format(str(int(n//1000)+1)))
  soup = BeautifulSoup(response, 'html.parser')
  if soup.find('a', text='SCP-{}'.format(num_as_str)) == None:
    # some contain the value but don't have sibling eg start of block of 100  can try and catch or tweak
    # can try next li after previous num find? assuming there won't be any consecutive non standard titles...
    if soup.find('a', text='SCP-{}'.format(str(n-1))) != None and soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling() != None:
      title = soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling().text.split(" - ")[-1]
    else:
      title = soup.find('a', text='SCP-{}'.format(str(n+1))).parent.find_previous_sibling().text
      if ' - ' in title:
        title = title.split(" - ")[-1]
    print(title)
    # and here can also change/note the other code? if later want to add another column for these edge cases 
    # eg the ozymandias effect is both scp2235 and ura9611
  else:
    title = soup.find('a', text='SCP-{}'.format(num_as_str)).parent.text.split(" - ")[-1]
  
  # print(rating)
  # quotes to preserve commas and not cause csv issues
  info = ['SCP-{}'.format(num_as_str), "\"{}\"".format(title), "\"{}\"".format(text), url, rating]
  scp_main_df.loc[len(scp_main_df)] = info



In [ ]:
scp_main_df.head

In [67]:
scp_main_df.to_csv('scp6999.csv', mode='a', header=False) #should have done no index here as well

In [104]:
scp_df = pd.read_csv('scp6999.csv', delimiter=',', quotechar='"')
scp_df.columns = ['index', 'code', 'title', 'text', 'link', 'rating']


In [105]:
# some extra light  processing
scp_df['text'] = scp_df['text'].apply(lambda x: "\"{}\"".format(" ".join(i[1:-1] for i in x[2:-2].split(','))))


In [ ]:
import numpy as np
# can also quick check if all ratings start with + and remove if so for easier comparisons, sorting etc
# df auto switched to float, can do back to int and see if changes again... welp
scp_df['rating'] = scp_df['rating'].apply(lambda x: int(x) if np.isnan(x)==False else x)
           
# maybe add a flag for ones that are inaccessible/deleted (title might be easy way to filter )
scp_df.head

In [107]:
scp_df.to_csv('scpjoinedtext.csv')

In [ ]:
print(scp_df['text'][1])
# may still have some issues with the join method?